In [1]:
import os
import sys
import matplotlib.pyplot as plt
from pyproj import CRS
from pyresample import geometry
import numpy as np


sys.path.insert(0, '/home/cameron/Projects/hypso-package')

# Import Satellite Object
from hypso import Hypso1

In [2]:
dir_path = '/home/cameron/Dokumenter/Data/frohavet'
nc_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-l1a.nc')
points_file = os.path.join(dir_path, 'frohavet_2024-05-06_1017Z-bin3.points')

In [3]:
satobj = Hypso1(hypso_path=nc_file, points_path=points_file, verbose=True)

#satobj.generate_l1b_cube()

[INFO] Capture spatial dimensions: (598, 1092)
[INFO] Capture capture type: wide
[INFO] Running georeferencing...
No image mode provided. Detected image mode: bin3
[INFO] Using UTM map: WGS 84 / UTM zone 32N EPSG: 32632


In [4]:
satobj.resolution

592.8625951680069

In [5]:
l1a_scene = satobj.get_l1a_satpy_scene()
#l1b_scene = satobj.get_l1b_satpy_scene()

In [7]:
#l1a_scene._datasets.keys()

In [8]:
l1a_scene.coarsest_area()

TypeError: type NoneType doesn't define __round__ method

Shape: (598, 1092)
Lons: <xarray.DataArray (y: 598, x: 1092)>
array([[ 9.95151937,  9.95278019,  9.95404101, ..., 11.32704514,
        11.32831054, 11.32957594],
       [ 9.94718315,  9.94844373,  9.94970432, ..., 11.32244953,
        11.32371469, 11.32497986],
       [ 9.94284817,  9.94410852,  9.94536887, ..., 11.31785518,
        11.3191201 , 11.32038503],
       ...,
       [ 7.59193217,  7.59305127,  7.59417037, ...,  8.81312522,
         8.8142489 ,  8.81537258],
       [ 7.58833825,  7.58945711,  7.59057598, ...,  8.80927192,
         8.81039536,  8.81151881],
       [ 7.58474558,  7.58586421,  7.58698284, ...,  8.80541987,
         8.80654307,  8.80766628]])
Dimensions without coordinates: y, x
Lats: <xarray.DataArray (y: 598, x: 1092)>
array([[65.5340918 , 65.53395482, 65.53381781, ..., 65.36610637,
        65.3659348 , 65.3657632 ],
       [65.52914767, 65.52901068, 65.52887366, ..., 65.36115259,
        65.36098102, 65.36080941],
       [65.52420333, 65.52406634, 65.52392931

In [9]:
satobj.along_track_mean_gsd

592.8625951680069

In [5]:
test = [1, None]

In [7]:
max([])

ValueError: max() arg is an empty sequence